In [48]:
import pandas as pd
import numpy as np
import logging
import multiprocessing

import sys
sys.path.append("../../src/common")
from pharmacy_common import PharmacyCommon
common = PharmacyCommon()

In [ ]:
train_test_validation_path = "../../data/train_test_validation_data/XO_train_test_validation_data.xlsx"
train_dataset = pd.read_excel(train_test_validation_path, sheet_name='train_dataset')
print(len(train_dataset))
train_dataset.head(10)

337


,CID,SMILES,IC50(nM),Type,substructure
0,145967694,CC1=CC2=C(C=C1)N=C(O2)/C(=N/O)/CC3=CC=CC=C3,17500.0,inactive,7
1,76329670,CC1(C=CC2=CC(=C(C=C2O1)O)C(=O)/C=C/C3=CC(=C(C=...,1800.0,active,16
2,5320686,C1=CC(=CC=C1/C=C/C(=O)OC[C@@H]2[C@H]([C@@H]([C...,100000.0,inactive,6
3,155903284,C1=CC(=CC=C1C2=NC=NN2)NC(=O)C3C(NC(=O)NC3=O)O,1400.0,active,1
4,137648214,CCCCC1=NN2C(=N1)C3=C(NC2=O)NN=C3,529.0,active,9
5,156780202,C1=CC=C(C=C1)CN2C=C(C3=C2C=CC(=C3)NC(=O)C4=CC(...,820.0,active,3
6,135463610,CCCCSC1=NC(=C(C(=O)N1)NC=O)N,100000.0,inactive,3
7,156705027,C1=CC(=CC=C1COC2=C(C=C(C=C2)C3=NC=C4C(=N3)NNC4...,151.0,active,3
8,162680022,C1=CN=C(C=C1NC(=O)C2C(NC(=O)NC2=O)O)C(=O)NO,73800.0,inactive,7
9,24896701,CCC[C@H](C[C@@H]1[C@@H](C2=CC(=C(C(=C2C(=O)O1)...,612000.0,inactive,2


In [50]:
all_screening_data_path = "../../data/preprocessed_data/preprocessed_screening_dataset.xlsx"
all_screening_data = pd.read_excel(all_screening_data_path)
print(len(all_screening_data))
all_screening_data.head(10)

45


,Code,SMILES,Molecular Weight
0,BS-1,OC[C@@H](O1)[C@@H](O)[C@H](O)[C@]1(CO)O[C@@H](...,342.116212
1,BS-10,OC(C(C(C(COC(/C=C/C1=CC=C(O)C=C1)=O)O2)O)O)C2O...,478.111126
2,BS-11,O=C1CC(C2=CC=C(O)C(O)=C2)OC3=C1C(O)=CC(OC4[C@@...,450.116212
3,BS-12,COC1=C(O[C@H]2[C@H](O)[C@@H](O)[C@H](O)[C@@H](...,522.210112
4,BS-13,O=C[C@@H]([C@H]([C@@H]([C@@H](COC(C1=CC(O)=C(O...,940.118181
5,BS-14,O[C@@H]([C@@H]([C@@H](COC(C1=CC(O)=C(O)C(O)=C1...,630.122085
6,BS-15,O=C1C=C(C2=CC=C(O)C(O)=C2)OC3=C1C(O)=CC(OC4[C@...,448.100561
7,BS-16,COC1=C(O)C=C2[C@@H](C3=CC(OC)=C(O)C=C3)[C@@H](...,360.157288
8,BS-17,O=C1OC2=C(C3=C1C=C(O)C(O)=C3OC4=O)C4=CC(O)=C2O,302.006267
9,BS-18,O=C(O)[C@H](OC(/C=C/C1=CC(O)=C(O)C=C1)=O)CC2=C...,360.084517


In [51]:
def cal_tc(array1, array2):
    if len(array1) != len(array2):
        raise ValueError("The arrays must have the same length.")
    
    # Calculate the Tanimoto coefficient
    intersection = sum(a and b for a, b in zip(array1, array2))
    union = sum(a or b for a, b in zip(array1, array2))
    
    if union == 0:  # Handle the case when both arrays are all zeros
        return 0.0
    else:
        tanimoto_coefficient = intersection / union
        return tanimoto_coefficient


In [52]:
def calculate_distances(identifier, screening_vectors, X_Train):
    distances = []
    for screening_vector in screening_vectors:
        tc_dist = np.sum(screening_vector & X_Train, axis=1) / np.sum(screening_vector | X_Train, axis=1)
        distances.append(tc_dist)
    return identifier, distances

def find_nearest_neighbors_distance(X_Screening, X_Train, n_neighbors):
    nearest_neighbors_distances = []
    nearest_neighbors_indices = []
    X_Screening = np.array(X_Screening)
    X_Train = np.array(X_Train)
    if(np.size(X_Screening) == 0):
        return nearest_neighbors_distances, nearest_neighbors_indices
     
    if X_Screening.shape[1] != X_Train.shape[1]:
        raise ValueError("X_Screening bit vectors must have the same size as X_Train bit vectors: " + str(X_Train.shape[1]))

    num_processes = 6
    
    if len(X_Screening) <= num_processes:
        screening_chunks = [(i, X_Screening[i:i + 1]) for i in range(len(X_Screening))]
    else:
        chunk_size = len(X_Screening) // num_processes
        screening_chunks = [(i, X_Screening[i:i + chunk_size]) for i in range(0, len(X_Screening), chunk_size)]

    pool = multiprocessing.Pool(processes=num_processes)
    results = pool.starmap(calculate_distances, [(i, chunk, X_Train) for i, chunk in screening_chunks])
    pool.close()
    pool.join()

    # Sort the results by identifier to ensure the correct order
    results.sort(key=lambda x: x[0])

    # Extract the distances and indices
    for _, distances in results:
        for distance in distances:
            # Get the indices of the first n_neighbors elements with the largest Tanimoto coefficients
            nearest_neighbor_indices = np.argsort(distance)[::-1][:n_neighbors]

            # Extract the distances to the nearest neighbors
            nearest_neighbors_distances.append([distance[i] for i in nearest_neighbor_indices])
            nearest_neighbors_indices.append(nearest_neighbor_indices)

    return nearest_neighbors_distances, nearest_neighbors_indices

In [53]:
def update_average_distance(screening_data, n_neighbors, X_train, bits):
    logging.info(f"[+] Update average distance for screening dataset")
    working_dataset = screening_data
    if(len(working_dataset)>0):
        X_Screening = common.gen_ecfp4_fpts(working_dataset["SMILES"], bits)
        # X_Screening = common.gen_maccs_fpts(working_dataset["SMILES"])
        logging.info(f"[-] Start finding nearest neighbor for: {len(X_Screening)}")
        #Find nearest neighbor
        dist_array, nn_idx = find_nearest_neighbors_distance(X_Screening=X_Screening, n_neighbors=n_neighbors, X_Train=X_train)
        result_df = pd.DataFrame({'Code': working_dataset['Code'],'SMILES': working_dataset['SMILES'],
                                  'Molecular Weight':working_dataset['Molecular Weight'] ,
                                 'AVG_DISTANCE': np.average(dist_array, axis=1)})
    else:
        logging.info("[-] Empty data, skip this batch!")
        print("[-] Empty data, skip this batch!")    
    return result_df

In [55]:
screen_dataset = all_screening_data

In [56]:
fpt_bits = 1024
# X_Train = common.gen_maccs_fpts(train_dataset['SMILES'])
X_Train = common.gen_ecfp4_fpts(train_dataset['SMILES'],bits = fpt_bits)

# nn_nums: numbers of nearest-neighbors
## AD for MACCS, ECFP4 1024bits
nn_nums = 5
result= update_average_distance(screening_data=screen_dataset, n_neighbors=nn_nums,X_train = X_Train, bits= fpt_bits)

Progress:   0%|          | 0/337 [00:00<?, ?it/s][16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:

[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerator
[16:20:02] DEPRECATION WARNING: please use MorganGenerat

In [57]:
result.head(10)

,Code,SMILES,Molecular Weight,AVG_DISTANCE
0,BS-1,OC[C@@H](O1)[C@@H](O)[C@H](O)[C@]1(CO)O[C@@H](...,342.116212,0.316328
1,BS-10,OC(C(C(C(COC(/C=C/C1=CC=C(O)C=C1)=O)O2)O)O)C2O...,478.111126,0.434411
2,BS-11,O=C1CC(C2=CC=C(O)C(O)=C2)OC3=C1C(O)=CC(OC4[C@@...,450.116212,0.474506
3,BS-12,COC1=C(O[C@H]2[C@H](O)[C@@H](O)[C@H](O)[C@@H](...,522.210112,0.386835
4,BS-13,O=C[C@@H]([C@H]([C@@H]([C@@H](COC(C1=CC(O)=C(O...,940.118181,0.237275
5,BS-14,O[C@@H]([C@@H]([C@@H](COC(C1=CC(O)=C(O)C(O)=C1...,630.122085,0.430236
6,BS-15,O=C1C=C(C2=CC=C(O)C(O)=C2)OC3=C1C(O)=CC(OC4[C@...,448.100561,0.675677
7,BS-16,COC1=C(O)C=C2[C@@H](C3=CC(OC)=C(O)C=C3)[C@@H](...,360.157288,0.378021
8,BS-17,O=C1OC2=C(C3=C1C=C(O)C(O)=C3OC4=O)C4=CC(O)=C2O,302.006267,0.302106
9,BS-18,O=C(O)[C@H](OC(/C=C/C1=CC(O)=C(O)C=C1)=O)CC2=C...,360.084517,0.466401


In [61]:
#write to file excel
# all_result.to_excel("../../results/average_distance/avg_distance_screening_dataset_MACCS.xlsx", index=False)
result.to_excel("../../results/average_distance/avg_distance_screening_dataset_ECFP4_1024bits.xlsx", index=False)